# Support functions of the package OptMSP

In [1]:
### Import packages
import pandas as pd
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import itertools
from collections import Counter

# For plotting
import seaborn as sns
sns.set_theme(style="ticks")
sns.axes_style("darkgrid")
sns.set_theme()

from IPython.display import display, HTML

# For storing DataFrames
import pickle

import sys
from os import getcwd
sys.path.append(getcwd())
import MultiStagePackage.OptMSPfunctions as msp


from itertools import islice
import collections

## Support functions for ```doBruteForceAna()``` and ```doBruteForceNum()``` for generating time switches

### ```doTswitches()```

- ```doTswitches(n_stages, t_start, t_end, min_duration, density)```
    - ```n_stages``` = max number of stages the user wants to analyze (e.g. ```2``` for 2-stage)
    - ```t_start``` = start time (e.g. ```0```)
    - ```t_end``` = end time (e.g. ```24```)
    - ```min_duration``` = minimum duration for a stage (e.g. ```1```, meaning that all stages are at least 1 hour active)
    - ```density``` = density of time switches (e.g. ```1``` -> time switches are only tested at full hours; ```2``` -> time switches can occur every half hour); default=1
    - **OUTPUT**= returns a list including arrays containing for each transition of stages the possible tswitch time points 

In [2]:
# Example 1: 3-Stage with process time from 0-10 hours with each stage lasting at least 2 hours and switching times only at full hours
n_stages = 3
t_start = 0
t_end = 10
min_duration = 2
density = 1

# Output: List with two arrays (first array for transition of 1. to 2. stage and second array for transition of 2. to 3. stage)
msp.doTswitches(n_stages, t_start, t_end, min_duration, density)

[array([2., 3., 4., 5., 6.]), array([4., 5., 6., 7., 8.])]

In [3]:
# Example 2: 2-Stage with process time from 0-24 hours with each stage lasting at least 1 hour and switching times only at full hours
n_stages = 2
t_start = 0
t_end = 24
min_duration = 1
density = 1
 
msp.doTswitches(n_stages, t_start, t_end, min_duration, density)

[array([ 1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12., 13.,
        14., 15., 16., 17., 18., 19., 20., 21., 22., 23.])]

In [4]:
# Example 3: 2-Stage with process time from 0-24 hours with each stage lasting at least 30min and switching times possible every half hour
n_stages = 2
t_start = 0
t_end = 24
min_duration = 0.5
density = 2

msp.doTswitches(n_stages, t_start, t_end, min_duration, density)

[array([ 0.5,  1. ,  1.5,  2. ,  2.5,  3. ,  3.5,  4. ,  4.5,  5. ,  5.5,
         6. ,  6.5,  7. ,  7.5,  8. ,  8.5,  9. ,  9.5, 10. , 10.5, 11. ,
        11.5, 12. , 12.5, 13. , 13.5, 14. , 14.5, 15. , 15.5, 16. , 16.5,
        17. , 17.5, 18. , 18.5, 19. , 19.5, 20. , 20.5, 21. , 21.5, 22. ,
        22.5, 23. , 23.5])]

WARNING: 

In some cases (e.g. see below in example 4), the tswitches are incorrect. This is explained by the ```linspace``` function and the parameters that are used. In the case below the ```min_duration``` parameter (each stage lasts at least **30min**) is **"smaller"** than the ```density``` parameter (density=1 means switching times are possible **every hour**). In a 3-Stage setup, this means that the process time from 0-24 hours cannot be split equally by ```linspace```. 
The problem can be resolved by adjusting the parameters such that ```density```<= ```min_duration``` (please note that in this context the meaning of density is meant e.g. density=4 means 0.25h --> ***```density```=4*** <= ***```min_duration```=0.25***)

Examples in case of a 3-Stage approach: 
| density         | min_duration | Correct split of time? |
|-----------------|--------------|------------------------|
| 1 (every 1hour) | 0.5 (30min)  | no                     |
| 2 (every 30min) | 0.5 (30min)  | yes                    |
| 4 (every 15min) | 0.5 (30min)  | yes                    |
| 2 (every 30min) | 0.25 (15min) | no                     |
| 4 (every 15min) | 0.25 (15min) | yes                    |

In [5]:
# Example 4: 3-Stage with process time from 0-24 hours with each stage lasting at least 30min and switching times possible every hour
# This example represents an incorrect split of tswitches!
n_stages = 3
t_start = 0
t_end = 24
min_duration = 0.5
density = 1

msp.doTswitches(n_stages, t_start, t_end, min_duration, density)

[array([ 0.5       ,  1.52272727,  2.54545455,  3.56818182,  4.59090909,
         5.61363636,  6.63636364,  7.65909091,  8.68181818,  9.70454545,
        10.72727273, 11.75      , 12.77272727, 13.79545455, 14.81818182,
        15.84090909, 16.86363636, 17.88636364, 18.90909091, 19.93181818,
        20.95454545, 21.97727273, 23.        ]),
 array([ 1.        ,  2.02272727,  3.04545455,  4.06818182,  5.09090909,
         6.11363636,  7.13636364,  8.15909091,  9.18181818, 10.20454545,
        11.22727273, 12.25      , 13.27272727, 14.29545455, 15.31818182,
        16.34090909, 17.36363636, 18.38636364, 19.40909091, 20.43181818,
        21.45454545, 22.47727273, 23.5       ])]

### ```doTswitchesCombis()```

- ```doTswitchesCombis(t_switches, min_duration)```
    - ```t_switches``` = Output from ```doTswitches()```
    - ```min_duration``` = minimum duration for a stage (e.g. `1`, meaning that all stages are at least 1 hour active)
    - **OUTPUT**= returns a list of all possible combinations of tswitches

In [6]:
# For a 2-Stage there is only on tswitch
msp.doTswitchesCombis(msp.doTswitches(n_stages=2, t_start=0, t_end=24, min_duration=0.5, density=1), min_duration=0.5)

[(0.5,),
 (1.5,),
 (2.5,),
 (3.5,),
 (4.5,),
 (5.5,),
 (6.5,),
 (7.5,),
 (8.5,),
 (9.5,),
 (10.5,),
 (11.5,),
 (12.5,),
 (13.5,),
 (14.5,),
 (15.5,),
 (16.5,),
 (17.5,),
 (18.5,),
 (19.5,),
 (20.5,),
 (21.5,),
 (22.5,),
 (23.5,)]

In [7]:
# For a 3-Stage there are two tswitches for each combination
msp.doTswitchesCombis(msp.doTswitches(n_stages=3, t_start=0, t_end=24, min_duration=0.5, density=2), min_duration=0.5)

[(0.5, 1.0),
 (0.5, 1.5),
 (0.5, 2.0),
 (0.5, 2.5),
 (0.5, 3.0),
 (0.5, 3.5),
 (0.5, 4.0),
 (0.5, 4.5),
 (0.5, 5.0),
 (0.5, 5.5),
 (0.5, 6.0),
 (0.5, 6.5),
 (0.5, 7.0),
 (0.5, 7.5),
 (0.5, 8.0),
 (0.5, 8.5),
 (0.5, 9.0),
 (0.5, 9.5),
 (0.5, 10.0),
 (0.5, 10.5),
 (0.5, 11.0),
 (0.5, 11.5),
 (0.5, 12.0),
 (0.5, 12.5),
 (0.5, 13.0),
 (0.5, 13.5),
 (0.5, 14.0),
 (0.5, 14.5),
 (0.5, 15.0),
 (0.5, 15.5),
 (0.5, 16.0),
 (0.5, 16.5),
 (0.5, 17.0),
 (0.5, 17.5),
 (0.5, 18.0),
 (0.5, 18.5),
 (0.5, 19.0),
 (0.5, 19.5),
 (0.5, 20.0),
 (0.5, 20.5),
 (0.5, 21.0),
 (0.5, 21.5),
 (0.5, 22.0),
 (0.5, 22.5),
 (0.5, 23.0),
 (0.5, 23.5),
 (1.0, 1.5),
 (1.0, 2.0),
 (1.0, 2.5),
 (1.0, 3.0),
 (1.0, 3.5),
 (1.0, 4.0),
 (1.0, 4.5),
 (1.0, 5.0),
 (1.0, 5.5),
 (1.0, 6.0),
 (1.0, 6.5),
 (1.0, 7.0),
 (1.0, 7.5),
 (1.0, 8.0),
 (1.0, 8.5),
 (1.0, 9.0),
 (1.0, 9.5),
 (1.0, 10.0),
 (1.0, 10.5),
 (1.0, 11.0),
 (1.0, 11.5),
 (1.0, 12.0),
 (1.0, 12.5),
 (1.0, 13.0),
 (1.0, 13.5),
 (1.0, 14.0),
 (1.0, 14.5),
 (1.0, 15.0),

### ```doCreateTimesList()```
This function incorporates ```doTswitches``` and ```doTswitchesCombis()```.

- ```doCreateTimesList(n_stages, t_start, t_end, min_duration, density)```
    - ```n_stages``` = max number of stages the user wants to analyze (e.g. ```2``` for 2-stage)
    - ```t_start``` = start time (e.g. ```0```)
    - ```t_end``` = end time (e.g. ```24```)
    - ```min_duration``` = minimum duration for a stage (e.g. ```1```, meaning that all stages are at least 1 hour active)
    - ```density``` = density of time switches (e.g. ```1``` -> time switches are only tested at full hours; ```2``` -> time switches can occur every half hour); default=1
    - **OUTPUT**= returns a list of all possible combinations of tswitches and includes start and end time in each array

In [8]:
msp.doCreateTimesList(n_stages=3, t_start=0, t_end=10, min_duration=0.5, density=2)

[(0.0, 0.5, 1.0, 10.0),
 (0.0, 0.5, 1.5, 10.0),
 (0.0, 0.5, 2.0, 10.0),
 (0.0, 0.5, 2.5, 10.0),
 (0.0, 0.5, 3.0, 10.0),
 (0.0, 0.5, 3.5, 10.0),
 (0.0, 0.5, 4.0, 10.0),
 (0.0, 0.5, 4.5, 10.0),
 (0.0, 0.5, 5.0, 10.0),
 (0.0, 0.5, 5.5, 10.0),
 (0.0, 0.5, 6.0, 10.0),
 (0.0, 0.5, 6.5, 10.0),
 (0.0, 0.5, 7.0, 10.0),
 (0.0, 0.5, 7.5, 10.0),
 (0.0, 0.5, 8.0, 10.0),
 (0.0, 0.5, 8.5, 10.0),
 (0.0, 0.5, 9.0, 10.0),
 (0.0, 0.5, 9.5, 10.0),
 (0.0, 1.0, 1.5, 10.0),
 (0.0, 1.0, 2.0, 10.0),
 (0.0, 1.0, 2.5, 10.0),
 (0.0, 1.0, 3.0, 10.0),
 (0.0, 1.0, 3.5, 10.0),
 (0.0, 1.0, 4.0, 10.0),
 (0.0, 1.0, 4.5, 10.0),
 (0.0, 1.0, 5.0, 10.0),
 (0.0, 1.0, 5.5, 10.0),
 (0.0, 1.0, 6.0, 10.0),
 (0.0, 1.0, 6.5, 10.0),
 (0.0, 1.0, 7.0, 10.0),
 (0.0, 1.0, 7.5, 10.0),
 (0.0, 1.0, 8.0, 10.0),
 (0.0, 1.0, 8.5, 10.0),
 (0.0, 1.0, 9.0, 10.0),
 (0.0, 1.0, 9.5, 10.0),
 (0.0, 1.5, 2.0, 10.0),
 (0.0, 1.5, 2.5, 10.0),
 (0.0, 1.5, 3.0, 10.0),
 (0.0, 1.5, 3.5, 10.0),
 (0.0, 1.5, 4.0, 10.0),
 (0.0, 1.5, 4.5, 10.0),
 (0.0, 1.5, 5.0,

In [9]:
msp.doCreateTimesList(n_stages=3, t_start=0, t_end=24, min_duration=1, density=4)

[(0.0, 1.0, 2.0, 24.0),
 (0.0, 1.0, 2.25, 24.0),
 (0.0, 1.0, 2.5, 24.0),
 (0.0, 1.0, 2.75, 24.0),
 (0.0, 1.0, 3.0, 24.0),
 (0.0, 1.0, 3.25, 24.0),
 (0.0, 1.0, 3.5, 24.0),
 (0.0, 1.0, 3.75, 24.0),
 (0.0, 1.0, 4.0, 24.0),
 (0.0, 1.0, 4.25, 24.0),
 (0.0, 1.0, 4.5, 24.0),
 (0.0, 1.0, 4.75, 24.0),
 (0.0, 1.0, 5.0, 24.0),
 (0.0, 1.0, 5.25, 24.0),
 (0.0, 1.0, 5.5, 24.0),
 (0.0, 1.0, 5.75, 24.0),
 (0.0, 1.0, 6.0, 24.0),
 (0.0, 1.0, 6.25, 24.0),
 (0.0, 1.0, 6.5, 24.0),
 (0.0, 1.0, 6.75, 24.0),
 (0.0, 1.0, 7.0, 24.0),
 (0.0, 1.0, 7.25, 24.0),
 (0.0, 1.0, 7.5, 24.0),
 (0.0, 1.0, 7.75, 24.0),
 (0.0, 1.0, 8.0, 24.0),
 (0.0, 1.0, 8.25, 24.0),
 (0.0, 1.0, 8.5, 24.0),
 (0.0, 1.0, 8.75, 24.0),
 (0.0, 1.0, 9.0, 24.0),
 (0.0, 1.0, 9.25, 24.0),
 (0.0, 1.0, 9.5, 24.0),
 (0.0, 1.0, 9.75, 24.0),
 (0.0, 1.0, 10.0, 24.0),
 (0.0, 1.0, 10.25, 24.0),
 (0.0, 1.0, 10.5, 24.0),
 (0.0, 1.0, 10.75, 24.0),
 (0.0, 1.0, 11.0, 24.0),
 (0.0, 1.0, 11.25, 24.0),
 (0.0, 1.0, 11.5, 24.0),
 (0.0, 1.0, 11.75, 24.0),
 (0.0, 1.0, 

In [10]:
## You can also check how many tswitch combinations exist for your own process:
len(msp.doCreateTimesList(n_stages=2, t_start=0, t_end=24, min_duration=1, density=4))

89

### ```doCountCombs()```
This function counts all to be tested Model/tswitch-combinations.

- ```doCountCombs(combi, n_stages, t_start, t_end, min_duration, density)```
    - ```combi``` = combinations as list of tuples (e.g. ```[(0, 0), (0, 1), ... , (5, 5)]```) (can be generated by ```itertools.product```)
    - ```n_stages``` = max number of stages the user wants to analyze (e.g. ```2``` for 2-stage)
    - ```t_start``` = start time (e.g. ```0```)
    - ```t_end``` = end time (e.g. ```24```)
    - ```min_duration``` = minimum duration for a stage (e.g. ```1```, meaning that all stages are at least 1 hour active)
    - ```density``` = density of time switches (e.g. ```1``` -> time switches are only tested at full hours; ```2``` -> time switches can occur every half hour); default=1
    - **OUTPUT**= returns the total number of combinations

In [11]:
# Example 1: 2-Stage with process time from 0-24 hours with each stage lasting at least 1 hour and switching times possible quarter hour
combis = list(
    itertools.product([0, 1, 2, 3, 4, 5], [0, 1, 2, 3, 4, 5])
)  # all combinations for 2-stage: [(0, 0), (0, 1), ... , (5, 5)]

res = msp.doCountCombs(combis, n_stages=2, t_start=0, t_end=24, min_duration=1, density=4)
res

3204

In [12]:
# Example 2: 4-Stage with process time from 0-24 hours with each stage lasting at least 1 hour and switching times possible quarter hour
combis = list(
    itertools.product([0, 1, 2], [0, 1, 2], [0, 1, 2], [0, 1, 2])
)  

res = msp.doCountCombs(combis, n_stages=4, t_start=0, t_end=24, min_duration=1, density=4)
res

7442361

Note that not all of them are actually tested (and therefore will be not displayed in the results dataframe) because some processes will already end early. For example the substrate is consumed at a 2-Stage process after 10 hours (tswitch would be at 11 hours) -> all the following tswitches at e.g. 12 hours, 13 hours, ... will be skipped because the process already finished before the second stage started.